### [ 수치형과 범주형  관계 ]
- 범주형(또는 텍스트형) 변수와 수치형 변수 간의 관계 => 통계적 측도(association measure)
- 수치형 변수와 수치형 변수 간의 관계 => 상관계수

In [ ]:
## ------------------------------------------------------------
## 1) 모듈 로딩 및 데이터 준비
## ------------------------------------------------------------
import pandas as pd
import numpy as np
import sys
sys.path.append(r'C:\Users\qjadl\Desktop\KDT10\EX_DATA_VS\COMM')
from datautill import *

DATA_FILE = '../Data/titanic_train.csv'

## 데이터 로딩
dataDF = pd.read_csv(DATA_FILE)

## 본사본 준비
dataDF2 = dataDF.copy()


In [ ]:

## ------------------------------------------------------------
# 2) pandas/numpy만으로 Eta-squared(η²) 계산 함수
## ------------------------------------------------------------
## ANOVA(Analysis of Variance : 분산 분석)
## -> 여러 집단의 평균이 같은지 다른지를 검증하는 통계적 기법
## -> 여러 집단의 평균을 비교하기 위해 집단 내 분산과 집단 간 분산 비교
## -> 공식: η² = SS_between / SS_total
## -> 
## 함수기능 : 수치형 컬럼에 대한 범주형 컬럼의 영향 정도 반환
##           
## 함수이름 : eta_squared_np
## 매개변수 : df         : 데이터프레임
##           cat_col    : 범주형/테스트 컬럼
##           num_col    : 수치형 컬럼
## 결과반환 : 0 ~ 1 
## ------------------------------------------------------------
def eta_squared_np(df, cat_col, num_col):

    tmp = df[[cat_col, num_col]].dropna()  # 범주/수치 모두 결측 제거
    if tmp.empty:
        return np.nan, np.nan, np.nan, 0
    
    # 수치형 컬럼 값과 전체 평균
    y = tmp[num_col].to_numpy()
    overall_mean = y.mean()

    # 범주형 기준 그룹 평균/크기
    grouped = tmp.groupby(cat_col)[num_col]
    counts = grouped.size().to_numpy()
    means  = grouped.mean().to_numpy()

    # 집단간 제곱합 (SS_between) = Σ n_g * (μ_g - μ)^2
    ss_between = np.sum(counts * (means - overall_mean) ** 2)

    # 총 제곱합 (SS_total) = Σ (y_i - μ)^2
    ss_total   = np.sum((y - overall_mean) ** 2)

    # (선택) 집단내 제곱합 (SS_within) = SS_total - SS_between
    ss_within  = ss_total - ss_between

    ## 수치형 컬럼들 평균 계산
    y = tmp[num_col].to_numpy()
    overall_mean = y.mean()


    eta2 = ss_between / ss_total if ss_total > 0 else np.nan
    return float(eta2), float(ss_between), float(ss_total), int(len(counts))






In [ ]:
## ------------------------------------------------------------
# 3) 타겟과 후보 범주형 컬럼들
## ------------------------------------------------------------
num_col = "Age"
cat_cols = ["Pclass", "Sex", "Embarked"]

rows = []
for c in cat_cols:
    eta2, ssb, sst, k = eta_squared_np(dataDF2, c, num_col)
    rows.append({
        "cat_col": c,
        "eta_squared": eta2,
        "SS_between": ssb,
        "SS_total": sst,
        "num_groups": k
    })

for r in rows:
    print(r['cat_col'], r['eta_squared'])